In [ ]:
"""
Creating cloudwatch dashboards can be tedious and error prone.
This example provides a foundation on how to use nab3 to create a standardized cloudwatch dashboard in seconds.
It's repeatable, extendable and easy to adjust.
"""

import json

from double_click import echo

from nab3 import AWS as NabAWS

AWS = NabAWS()

In [ ]:
SERVICE_NAME = 'sample'
EXCLUDED_ECS_SERVICES = []
ELASTICACHE_CLUSTER = SERVICE_NAME
RDS_CLUSTER = None

if RDS_CLUSTER:
    try:
        await AWS.rds_cluster.get(id=RDS_CLUSTER)
    except:
        echo(f'{RDS_CLUSTER} not found. Setting RDS_CLUSTER = None')
        RDS_CLUSTER = None

if ELASTICACHE_CLUSTER:
    try:
        await AWS.elasticache_cluster.get(id=ELASTICACHE_CLUSTER)
    except:
        echo(f'{ELASTICACHE_CLUSTER} not found. Setting ELASTICACHE_CLUSTER = None')
        ELASTICACHE_CLUSTER = None

try:
    ALB = await AWS.load_balancer.get(name=SERVICE_NAME)
except:
    ALB = None

try:
    ECS_CLUSTER = await AWS.ecs_cluster.get(name=SERVICE_NAME, with_related=['services'])
except:
    ECS_CLUSTER = None

In [ ]:
dashboard_source = dict(widgets=[])
y = 0

In [ ]:
if ALB:
    # ALB Widgets
    widget_height = 9
    lb_id = ALB.arn.split(':loadbalancer/')[-1]
    dashboard_source['widgets'] += [
        {
                "type": "metric",
                "x": 0,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": [
                        [ "AWS/ApplicationELB", "RequestCount", "LoadBalancer", lb_id ],
                        [ ".", "HTTPCode_Target_2XX_Count", ".", "." ],
                        [ ".", "HTTPCode_Target_3XX_Count", ".", "." ],
                        [ ".", "HTTPCode_Target_4XX_Count", ".", "." ],
                        [ ".", "HTTPCode_Target_5XX_Count", ".", "." ]
                    ],
                    "view": "timeSeries",
                    "stacked": False,
                    "region": AWS.region,
                    "stat": "Sum",
                    "period": 300,
                    "title": "API Request Count"
                }
            },
        {
                "type": "metric",
                "x": 12,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": [
                        [ "AWS/ApplicationELB", "TargetResponseTime", "LoadBalancer", lb_id, { "label": "Max", "stat": "Maximum" } ],
                        [ "...", { "label": "Average" } ]
                    ],
                    "view": "timeSeries",
                    "stacked": False,
                    "region": AWS.region,
                    "title": "API Response Time",
                    "stat": "Average",
                    "period": 300
                }
            }
    ]
    
    y += widget_height

In [ ]:
if ECS_CLUSTER:
    # ECS Cluster Widgets
    widget_height = 6
    dashboard_source['widgets'] += [
            {
                "type": "metric",
                "x": 0,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": [
                        [ "AWS/ECS", "CPUUtilization", "ClusterName", ECS_CLUSTER.name, { "label": "Max", "stat": "Maximum" } ],
                        [ "...", { "label": "Average" } ]
                    ],
                    "view": "timeSeries",
                    "stacked": False,
                    "region": AWS.region,
                    "title": "Cluster CPU Utilization",
                    "stat": "Average",
                    "period": 300
                }
            },
            {
                "type": "metric",
                "x": 12,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": [
                        [ "AWS/ECS", "MemoryUtilization", "ClusterName", ECS_CLUSTER.name, { "label": "Max", "stat": "Maximum" } ],
                        [ "...", { "label": "Average" } ]
                    ],
                    "view": "timeSeries",
                    "stacked": False,
                    "region": AWS.region,
                    "title": "Cluster Memory Utilization",
                    "stat": "Average",
                    "period": 300
                }
            },
    ]
    
    y += widget_height

In [ ]:
if ECS_CLUSTER:
    # ECS Service Widget
    widget_height = 9
    cpu_metrics = []
    mem_metrics = []
    
    for ecs_service in ECS_CLUSTER.services:
        if ecs_service in EXCLUDED_ECS_SERVICES:
            continue
    
        cpu_metrics += [
            [ "AWS/ECS", "CPUUtilization", "ServiceName", ecs_service.name, "ClusterName", ECS_CLUSTER.name, { "label": f"{ecs_service.name} Max", "stat": "Maximum" } ],
            [ "...", { "label": f"{ecs_service.name} Average", "stat": "Average" } ]
        ]
    
        mem_metrics += [
            [ "AWS/ECS", "MemoryUtilization", "ServiceName", ecs_service.name, "ClusterName", ECS_CLUSTER.name, { "label": f"{ecs_service.name} Max", "stat": "Maximum" } ],
            [ "...", { "label": f"{ecs_service.name} Average", "stat": "Average" } ]
        ]
        
    
    dashboard_source['widgets'] += [
            {
                "type": "metric",
                "x": 0,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": cpu_metrics,
                    "view": "timeSeries",
                    "stacked": False,
                    "region": AWS.region,
                    "title": "Service CPU Utilization",
                    "stat": "Average",
                    "period": 300
                }
            },
            {
                "type": "metric",
                "x": 12,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": mem_metrics,
                    "view": "timeSeries",
                    "stacked": False,
                    "region": AWS.region,
                    "title": "Service Memory Utilization",
                    "stat": "Average",
                    "period": 300
                }
            },
    ]
    
    y += widget_height

In [ ]:
# Data store Widgets
widget_height = 3

if RDS_CLUSTER:
    dashboard_source['widgets'].append(
        {
            "type": "metric",
            "x": 0,
            "y": y,
            "width": 12,
            "height": widget_height,
            "properties": {
                "metrics": [
                    [ "AWS/RDS", "CPUUtilization", "DBClusterIdentifier", RDS_CLUSTER ],
                    [ ".", "Queries", ".", "." ],
                    [ ".", "DatabaseConnections", ".", "." ],
                    [ ".", "FreeableMemory", ".", "." ]
                ],
                "view": "singleValue",
                "region": AWS.region,
                "title": "RDB",
            }
        },
    )

if ELASTICACHE_CLUSTER:
    dashboard_source['widgets'].append(
            {
                "type": "metric",
                "x": 12 if RDS_CLUSTER else 0,
                "y": y,
                "width": 12,
                "height": widget_height,
                "properties": {
                    "metrics": [
                        [ "AWS/ElastiCache", "CPUUtilization", "CacheClusterId", ELASTICACHE_CLUSTER ],
                        [ ".", "CacheHitRate", ".", "." ],
                        [ ".", "BytesUsedForCache", ".", "." ],
                        [ ".", "CurrConnections", ".", "." ]
                    ],
                    "view": "singleValue",
                    "region": AWS.region,
                    "title": "Cache",
                }
            }
    )

y += widget_height  # In case additional widgets are added later

In [ ]:
with open(f'{SERVICE_NAME}_{AWS.region}_dashboard.json', 'w') as f:
    f.write(json.dumps(dashboard_source, indent=2))

